In [1]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994'
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [1]:
from transformers import GPT2Tokenizer, GPTNeoForCausalLM

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")
# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model= model.cuda()

In [4]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [5]:
pwd

'/workspace'

In [6]:
DATA_DIRECTORY = "workspace/lean_proof_recording/data"

In [7]:
with open("workspace/lean_proof_recording/data/cleaned_training_data/train.src") as f:
    train_src = f.readlines()

In [8]:
with open("workspace/lean_proof_recording/data/cleaned_training_data/train.tgt") as f:
    train_tgt = f.readlines()

In [9]:
ls workspace/lean_proof_recording/data/cleaned_training_data

data_and_metadata.csv  test.src     train.names  valid.index  valid.tgt
test.index             test.tgt     train.src    valid.names
test.names             train.index  train.tgt    valid.src


In [10]:
with open("workspace/lean_proof_recording/data/cleaned_training_data/valid.tgt") as f:
    val_tgt = f.readlines()

In [11]:
with open("workspace/lean_proof_recording/data/cleaned_training_data/valid.src") as f:
    val_src = f.readlines()

In [12]:
with open("workspace/lean_proof_recording/data/cleaned_training_data/test.src") as f:
    test_src = f.readlines()

In [13]:
with open("workspace/lean_proof_recording/data/cleaned_training_data/test.tgt") as f:
    test_tgt = f.readlines()

In [14]:
len(test_tgt)

7291

In [15]:
import torch

In [16]:
from torch.utils.data import Dataset
class TacticDataset(Dataset):
    def __init__(self, txt_list,tgt_list, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt,txt2 in zip(txt_list,tgt_list):
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>'
                                        + ' <GOAL> ' + txt +  ' <PROOFSTEP> ' +  txt2+ 
                                        '<|endoftext|>',
                                        truncation=True,
                                        padding="max_length")
            
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)
#             break
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [17]:
# del val_src

In [18]:
import gc
gc.collect()

13

In [19]:

validDataset = TacticDataset(val_src,val_tgt,tokenizer)
testDataset = TacticDataset(test_src,test_tgt,tokenizer)

In [20]:
trainDataset = TacticDataset(train_src,train_tgt,tokenizer)

In [21]:
# x = TacticDataset(test_src,test_tgt,tokenizer)

In [22]:
# x[0][0].shape

In [23]:
from torch import nn
from transformers import Trainer
from  transformers import TrainingArguments, Trainer

In [ ]:
{
  "train_batch_size": 15,
  "fp16": {
    "enabled": true,
    "min_loss_scale": 1,
    "opt_level": "O2"
  },
  "zero_optimization": {
    "stage": 2,
    "offload_param": {
      "device": "cpu"
    },
    "offload_optimizer": {
      "device": "cpu"
    },
    "allgather_partitions": true,
    "allgather_bucket_size": 5e8,
    "contiguous_gradients": true
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 5e-05,
      "betas": [
        0.9,
        0.999
      ],
      "eps": 1e-08
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 5e-05,
      "warmup_num_steps": 50
    }
  }
}


In [24]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

In [25]:
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=5,
                                  logging_steps=5000,
                                  save_steps=20000,                                   
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  warmup_steps=100,
                                  weight_decay=0.01,  
                                  logging_dir='./logs')

In [26]:
import mpi4py

In [27]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=trainDataset,
                  eval_dataset=validDataset, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])})

In [58]:
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=5,
                                  logging_steps=5000,
                                  save_steps=20000,                                   
                                  per_device_train_batch_size=6,
                                  per_device_eval_batch_size=6,
                                  warmup_steps=100,
                                  weight_decay=0.01,  
                                  logging_dir='./logs',fp16= True)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [59]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=trainDataset,
                  eval_dataset=validDataset, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])})

Using amp half precision backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 168649
  Num Epochs = 5
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 140545


Step,Training Loss


In [ ]:
1+!